# Etsy Data
This project involves using selenium to web scrap data, and create a table.

We start by importing the necessary libraries.

Then we create productTable class, which first connects to the database (in this case mongodb), provides the user to choose the function to perform, creates a table if that is chosen, shows a product if that is chosen, or lists all products.

In [3]:
from selenium import webdriver
import time
import pymongo
import requests
from bson.objectid import ObjectId

In [6]:
class productTable():
    def __init__(self):
        print('x')
        self.client = pymongo.MongoClient("mongodb://localhost:27017")
        self.db = self.client["app"]
        self.collection = self.db["products"]
        self.listing = self.collection.find({"from":"etsy"})
        self.count = self.collection.count_documents({"from":"etsy"})
        
        self.choice()
    
    def choice(self):
        print("1- Add new product\n"  "2- Show a product\n" "3- List all products")
        choice = input("What do you want to do?:")

        if choice == '1':
            self.createTable()
        elif choice =='2':
            self.getProductbyId()
        elif choice =='3':
            self.listProducts()
        else:
            print("No such choice. Choose again")
            self.choice()

    def createTable(self):
        url = input("Link to the product: ")
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(2)
        productTitle = driver.title
        
        productPrice = driver.find_element('xpath',"//*[@id='listing-page-cart']/div[2]/div/div/div/p/text()" ).text
        productImage = driver.find_element('xpath',"//*[@id='photos']/div[3]/div/ul/li/img").get_attribute("src")
        time.sleep(2)
        driver.close()
        
        product = {
            "_id" : self.count +1,
            "name" : productTitle,
            "image" : productImage,
            "price" : productPrice,
            "from" : "etsy"
        }
        
        result = self.collection.insert_one(product)
        insertedProductId = result.inserted_id

        findInsertedProduct = self.collection.find({"_id" : ObjectId(insertedProductId)})
        for i in findInsertedProduct:
            
            print(i)
        
    
    def getProductbyId(self):
        desiredId = input("Enter Id: ")
        result = self.collection.find({"_id" : int(desiredId)})
        for i in result:
            print(i)


    def listProducts(self):
        result = self.collection.find()
        for i in result:
            print([i])


productTable()

x
1- Add new product
2- Show a product
3- List all products
{'_id': 1, 'name': 'FQ Pack of 5 Different Cutest Little Elephant in Blue 18 - Etsy Turkey', 'image': 'https://i.etsystatic.com/26992806/r/il/4164ba/3960713286/il_794xN.3960713286_6tpl.jpg', 'price': '207.26 TL', 'from': 'etsy'}


Note, that this project was written in 2021. The website now has protection against scraping.